In [1]:
import pandas as pd
import numpy as np
import regex as re
import geopy.distance

In [2]:
df = pd.read_csv('data_new.csv',index_col=False)

In [55]:
len(df[df['adres'].isnull()])

1378

In [12]:
len(df)

9580

In [4]:
df.drop_duplicates(inplace=True)

In [5]:
df = pd.read_csv('data_new.csv',index_col=False)
df = df[df['price']!='Zapytaj o cenę']
df = df[df['adres'].notnull()]
df = df.replace('Zapytaj','nieznane')
df = df.replace('brak informacji','nieznane')
df = df.drop(axis=1,columns=['availableSince','marketer'])

In [17]:
df.columns

Index(['price', 'size', 'formOfOwnership', 'numberOfRooms', 'condition',
       'floor', 'balcony/garden/terrace', 'rent', 'parkingSpot',
       'remoteService', 'heating', 'market', 'yearOfConstruction',
       'typeOfConstruction', 'windows', 'elevator', 'media', 'safetyMeasures',
       'equipment', 'additionalInfo', 'constructionMaterial',
       'coordinatesLink', 'adres'],
      dtype='object')

# Cena

In [58]:
def convertPriceCol(price) -> int:
    if(price == 'nieznane'):
        return 0
    if(price[-3:] == "EUR"):
        price = float(re.sub("\s|[^0-9,.]","",price).replace(",","."))    
        price = int(price*4.31)
        return price
    elif(price[-2:] == "zł"):
        price = int(float(re.sub("[^\d|\,]","",price).replace(",",".")))
        return price
    else:
        return price
df['price'] = df['price'].apply(convertPriceCol)

# Powierzchnia

In [18]:
def convertSizeCol(size):
    return float(re.sub("[^\d|\,]","",size).replace(",","."))
df['size'] = df['size'].apply(convertSizeCol)

df = df[(df['size'].between(10,500)) & (~df['size'].isin([420.00,470.00,460.00,347.70,379.00,303.00,317.00]))]

In [19]:
len(df)

9564

# Liczba pokoi

In [60]:
def convertNumberOfRooms(rooms) -> int:
    if(rooms == 'więcej niż 10'):
        return 11
    else:
        return int(rooms)
df['numberOfRooms'] = df['numberOfRooms'].apply(convertNumberOfRooms)   

# Stan

In [61]:
df['condition'].unique()

array(['do remontu', 'do zamieszkania', 'nieznane', 'do wykończenia'],
      dtype=object)

# Piętro

In [62]:
def convertFloor(floor) -> int:
    level = floor.split('/')
    if(level[0] == 'parter'):
        return 0
    elif(level[0] == 'suterena'):
        return -1
    elif(level[0] == 'nieznane' or (len(level) == 1 and level[0] == 'poddasze')):
        return -2
    elif(len(level)==1):
        return int(re.sub("[^\d]","",level[0]))
    if(len(level)==2):
        if(level[0] == 'poddasze'):
            return int(level[1]) 
        if(int(re.sub("[^\d]","",level[0]))>int(re.sub("[^\d]","",level[1]))):
            level[0] = level[1]
        return int(re.sub("[^\d]","",level[0]))


In [63]:
df['estateFloor'] = df['floor'].apply(convertFloor)

# Liczba pięter

In [64]:
def convertLastFloor(floor) -> int:        
    level = floor.split('/')
    if(len(level)==2):
        try:
            if(int(re.sub("[^\d]","",level[0]))>int(re.sub("[^\d]","",level[1]))):
                level[1] = level[0]
        except:
            1==1
        if(int(re.sub("[^\d]","",level[1]))>53):
            return -2
        return int(re.sub("[^\d]","",level[1]))
    return -2

df['lastFloor'] = df['floor'].apply(convertLastFloor)
df = df.drop(axis=1,columns=['floor'])

# Balkon / Ogród / terrace

In [65]:
def convertValues(row,column):
    values = list(map(lambda x: x.replace(" ", "_"), list(map(str.strip, row[f'{column}'].split(',')))))
    for val in values:
        if val != 'nieznane':
            row[f'{val}'] = int(1)
    return row   
df = df.apply(convertValues, column='balcony/garden/terrace',axis=1)
df = df.drop(axis=1,columns=['balcony/garden/terrace'])

# Czynsz

In [66]:
df['rent'] = df['rent'].apply(convertPriceCol)

# Miejsce parkingowe

In [67]:
df['parkingSpot'] = df['parkingSpot'].replace("nieznane",int(0))
df['parkingSpot'] = df['parkingSpot'].replace('garaż/miejsce parkingowe',(1))

# Ogrzewanie

In [68]:
print(df['heating'].unique())

['miejskie' 'nieznane' 'inne' 'gazowe' 'elektryczne' 'kotłownia'
 'piece kaflowe']


# Market

In [69]:
print(df['market'].unique())

['wtórny' 'pierwotny']


# Rok budowy

In [70]:
df.loc[df["yearOfConstruction"] == '1836', "yearOfConstruction"] = '1936'
def convertYear(year):
    if(len(year) == 10):
        return int(year[:4])
    if(year == 'nieznane'):
        return 0
    if(int(year) > 1800 and int(year) < 2027):
        return int(year)
    else:
        return int(0)
df['yearOfConstruction'] = df['yearOfConstruction'].apply(convertYear)
# df['yearOfConstruction'] = df['yearOfConstruction'].astype(int)

# Typ budynku

In [71]:
print(df['typeOfConstruction'].unique())

['blok' 'kamienica' 'apartamentowiec' 'nieznane' 'dom wolnostojący'
 'szeregowiec' 'plomba' 'loft']


# Okna

In [72]:
print(df['windows'].unique())

['nieznane' 'plastikowe' 'aluminiowe' 'drewniane']


# Winda

In [73]:
df['elevator'] = df['elevator'].replace("tak",int(1))
df['elevator'] = df['elevator'].replace('nie',int(0))

# Media

In [74]:
def convertValues(row,column):
    values = list(map(lambda x: x.replace(" ", "_"), list(map(str.strip, row[f'{column}'].split(',')))))
    for val in values:
        if val != 'nieznane':
            row[f'{val}'] = int(1)
    return row   
df = df.apply(convertValues, column='media',axis=1)
df = df.drop(axis=1,columns=['media'])

# Zabezpieczenia

In [75]:
df['additionalInfo'].unique()

array(['oddzielna kuchnia', 'oddzielna kuchnia, pom. użytkowe',
       'piwnica, oddzielna kuchnia', 'klimatyzacja', 'nieznane',
       'piwnica, pom. użytkowe', 'piwnica', 'pom. użytkowe',
       'klimatyzacja, dwupoziomowe, pom. użytkowe',
       'klimatyzacja, piwnica, dwupoziomowe', 'klimatyzacja, piwnica',
       'klimatyzacja, pom. użytkowe',
       'klimatyzacja, piwnica, oddzielna kuchnia',
       'klimatyzacja, piwnica, oddzielna kuchnia, pom. użytkowe',
       'dwupoziomowe', 'klimatyzacja, piwnica, pom. użytkowe',
       'klimatyzacja, oddzielna kuchnia, pom. użytkowe',
       'piwnica, oddzielna kuchnia, pom. użytkowe',
       'dwupoziomowe, pom. użytkowe', 'klimatyzacja, dwupoziomowe',
       'klimatyzacja, oddzielna kuchnia', 'piwnica, dwupoziomowe',
       'piwnica, oddzielna kuchnia, dwupoziomowe',
       'klimatyzacja, piwnica, dwupoziomowe, pom. użytkowe',
       'klimatyzacja, piwnica, oddzielna kuchnia, dwupoziomowe, pom. użytkowe',
       'oddzielna kuchnia, dwupoz

In [76]:
def convertValues(row,column) -> int:
    values = list(map(lambda x: x.replace(" ", "_"), list(map(str.strip, row[f'{column}'].split(',')))))
    for val in values:
        if val != 'nieznane':
            row[f'{val}'] = int(1)
    return row   
df = df.apply(convertValues, column='safetyMeasures',axis=1)
df = df.drop(axis=1,columns=['safetyMeasures'])

# Urządzenia

In [77]:
def convertValues(row,column) -> int:
    values = list(map(lambda x: x.replace(" ", "_"), list(map(str.strip, row[f'{column}'].split(',')))))
    for val in values:
        if val != 'nieznane':
            row[f'{val}'] = int(1)
    return row   
df = df.apply(convertValues, column='equipment',axis=1)
df = df.drop(axis=1,columns=['equipment'])

# Dodatkowe informacje

In [78]:

def convertValues(row,column) -> int:
    values = list(map(lambda x: x.replace(" ", "_"), list(map(str.strip, row[f'{column}'].split(',')))))
    for val in values:
        if val != 'nieznane':
            row[f'{val}'] = int(1)
    return row   
df = df.apply(convertValues, column='additionalInfo',axis=1)
df = df.drop(axis=1,columns=['additionalInfo'])

# Materiał 

# Współrzędne geograficzne

In [79]:
def getCoordinates(row):
    link = row['coordinatesLink']
    matches = re.search(r'll=([\d.]+),([\d.]+)', link)
    row['latitude'] = matches.group(1)
    row['longitude'] = matches.group(2)
    return row
df = df.apply(getCoordinates,axis=1)
df = df.drop(axis=1,columns=['coordinatesLink'])

# Adres 

In [20]:
def getAdres(row):
    values = list(map(str.strip, row['adres'].split(',')))
    if len(values) == 3 or len(values) == 4 and values[0] in ['Mysiadło','Dawidy','Blizne Łaszczyńskiego','Ząbki']:
        row['city'] = values[0]
    elif len(values) == 4 or len(values)==5 and values[1] in ['Bielawa','Raszyn','Blizne Łaszczyńskiego']:
        if any(x in values[2] for x in ['wołomiński','Raszyn','Stare Babice','wołomiński']):
            row['street'] = values[0]
            row['city'] = values[1]
        else:
            row['street'] = values[0]
            row['district'] = values[1]
            row['city'] = values[2]
    elif len(values) == 5:
        row['street'] = values[0]
        row['MSI'] = values[1]
        row['district'] = values[2]
        row['city'] = values[3]
    elif len(values) == 6:
        if any(x not in values[0] for x in ['ul','Hemara','Polska','Norwida','Wilcza','Dzieci']):
            row['street'] = values[1]
            row['MSI'] = values[2]
            row['district'] = values[3]
            row['city'] = values[4]
        else:
            row['street'] = values[0]
            row['MSI'] = values[2]
            row['district'] = values[3]
            row['city'] = values[4]
    return row
df = df.apply(getAdres,axis=1)
df = df.drop(axis=1,columns=['adres'])

df.drop_duplicates(subset=['street','size','yearOfConstruction'],inplace=True)

In [21]:
len(df)

7414

# Dystans od metra

In [81]:
metroCoordinates = [(52.131944, 21.065),(52.140833, 21.056389),(52.149167, 21.046389),
 (52.156111, 21.034722),(52.161944, 21.027778),(52.172778, 21.026389),
 (52.181389, 21.023056),(52.189722, 21.016667),(52.198611, 21.012222),
 (52.208611, 21.007778),(52.218333, 21.015278),(52.231389, 21.010556),
 (52.235833, 21.0075),(52.244722, 21.000556),(52.257778, 20.994444),
 (52.269167, 20.984167),(52.271667, 20.971667),(52.276389, 20.961389),
 (52.281944, 20.949444),(52.286389, 20.939444),(52.2911, 20.9289),
 (52.238611, 20.910278),(52.240278, 20.93),(52.2391, 20.9442),
 (52.2377, 20.9599), (52.23164, 20.967039), (52.230139, 20.983333),
 (52.233056, 20.998333), (52.235833, 21.0075), (52.237083, 21.017917),
 (52.24, 21.031667), (52.245833, 21.043333), (52.254722, 21.034722),
 (52.2634, 21.0453), (52.2695, 21.0514), (52.2753, 21.0554),
 (52.28369, 21.0622), (52.292111, 21.047361), (52.1736, 21.0146),
 (52.225278, 21.014167), (52.25, 20.998889)]
def getDistFromMetro(row):
    lat = row['latitude']
    lon = row['longitude']
    distFromMetros = []
    for metro in metroCoordinates:
        distFromMetros.append(geopy.distance.geodesic((lat,lon), metro).km)
    distFromMetros.sort()
    row['distFromMetro'] = distFromMetros[0]
    return row
df = df.apply(getDistFromMetro,axis=1)

In [82]:
len(df.columns)

48

In [ ]:
# exclude the most expensive apartment as its 4x the price of the previous one, 
# fix mismatch in "dzielnica" column
# apply average house year to the apartments with no years assigned
df = df[df['cena']<100000000]
df.loc[df["dzielnica"] == 'Bielawa', "dzielnica"] = "0"
df.loc[df["rokBudowy"] == 0, "rokBudowy"] = 2004

In [83]:
df = df[['price','rent','size','numberOfRooms','condition','constructionMaterial','typeOfConstruction',
         'windows','yearOfConstruction','estateFloor','lastFloor','dwupoziomowe','oddzielna_kuchnia','elevator',
         'piwnica','pom._użytkowe','balkon','ogródek','taras','internet','klimatyzacja','kuchenka',
         'lodówka','meble','piekarnik','pralka',
         'telefon', 'telewizja_kablowa','telewizor','zmywarka',
         'domofon_/_wideofon', 'drzwi_/_okna_antywłamaniowe','monitoring_/_ochrona',
         'system_alarmowy','rolety_antywłamaniowe','teren_zamknięty', 
         'market','formOfOwnership','heating','latitude','longitude','parkingSpot',
         'remoteService','street','MSI','district','city','distFromMetro']]
len(df.columns)
cols = ['cena','czynsz','rozmiar','liczbaPokoi','stan','materiał','typBudynku',
        'okna','rokBudowy','piętro','wysokośćBudynku','czyDwupoziomowe',
        'oddzielnaKuchnia','winda','piwnica','pomieszczenieUżytkowe','balkon','ogródek','taras','internet',
        'klimatyzacja','kuchenka','lodówka','meble','piekarnik','pralka','telefon',
        'telewizjaKablowa','telewizor','zmywarka','domofon','antywłamaniowe','monitoring/ochrona',
        'systemAlarmowy','roletyAntywłamaniowe','terenZamknęty',
        'rynek','typWłasności','ogrzewanie','długośćGeograficzna','szerokośćGeograficzna',
        'miejsceParkingowe','zdalnaObsługa','ulica','MSI','dzielnica','miasto',
        'odległośćOdMetra']
df.columns = cols

In [86]:
df['dzielnica'].unique()

array(['Ursynów', 'Białołęka', 'Wola', 'Śródmieście', 'Mokotów',
       'Praga-Północ', 'Praga-Południe', 'Bielany', 'Ochota', 'Wilanów',
       'Żoliborz', 'Targówek', 'Rembertów', 'Wesoła', 'Bemowo', 'Ursus',
       'Włochy', 'Wawer', 0, 'Bielawa'], dtype=object)

In [85]:
def fillCzynsz(row):
    if row['czynsz'] == 0:
        row['czynsz'] = row['rozmiar']*16
        return row
    return row
df = df.apply(fillCzynsz,axis=1)
def changeHeight(wys):
    if wys == 52:
        return 11
    return wys

df.replace(np.nan,0,inplace=True)
df[['czynsz','pomieszczenieUżytkowe','balkon','ogródek','taras','internet','klimatyzacja',
    'czyDwupoziomowe','oddzielnaKuchnia','piwnica', 
    'antywłamaniowe','monitoring/ochrona','systemAlarmowy',
    'kuchenka','lodówka', 'meble', 'piekarnik', 'pralka','telefon', 'telewizjaKablowa',
    'telewizor', 'zmywarka', 'domofon', 'roletyAntywłamaniowe', 'terenZamknęty']] = df[[
   'czynsz','pomieszczenieUżytkowe','balkon','ogródek','taras','internet','klimatyzacja','czyDwupoziomowe','oddzielnaKuchnia','piwnica' , 
    'antywłamaniowe','monitoring/ochrona','systemAlarmowy',
    'kuchenka','lodówka', 'meble', 'piekarnik', 'pralka','telefon', 'telewizjaKablowa',
    'telewizor', 'zmywarka', 'domofon', 'roletyAntywłamaniowe', 'terenZamknęty']].astype(int)
df.to_csv("formatted_data.csv",index=False)